In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
import os
import gc
from keras.preprocessing.image import image,load_img,img_to_array
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
#processing the image and converting them into numbers
#getting all images
train_pics = image.list_pictures("train/")
#test_pics = image.list_pictures("test/")

def image_resize(list,folder):
    #convert all to the same dimensions of 100*100
    os.mkdir(folder)
    i = 0
    for im in list:
        #get old name
        name = im.split("/")[1]
        try:
            #read the image
            z = load_img(im)
            #resize the image
            z = z.resize([32,32])
            #save the image
            z.save(folder+name)
            i += 1
            if (i %500 == 0 ):
                print("{} images converted".format(i))
        except:
            print("Cannot process {}".format(name))
    print("Conversion Completed")
print("training set")
image_resize(train_pics,"contrain/")
#print("testing set")
#image_resize(test_pics,"convtest/")

training set
500 images converted
1000 images converted
1500 images converted
2000 images converted
2500 images converted
3000 images converted
3500 images converted
4000 images converted
Conversion Completed


In [4]:
train = image.list_pictures("contrain/")
#test = image.list_pictures("convtest/")
#instatiate the test and train images
d1 = np.zeros([len(train),32,32,3])
#d2 = np.zeros([len(test),100,100,3])
def convert_to_numpy(file,list):
    i = 0
    for im in file:
        #read the image
        try:
            z = load_img(im)
            #convert into images and replace in the instatiated datasets above
            list[i] = img_to_array(z)
            i += 1
            if (i%500==0):
                print("{} images converted".format(i))
        except:
            print("Cannot process image {}".format(im))
    print ("Conversion is Completed")
print("Training Set")
convert_to_numpy(train,d1)
#print("Testing Set")
#convert_to_numpy(test,d2)

Training Set
500 images converted
1000 images converted
1500 images converted
2000 images converted
2500 images converted
3000 images converted
3500 images converted
4000 images converted
Conversion is Completed


In [5]:
# #normalizing the data
d1 = (d1 - 128)/ 128
#d2 = (d2 - 128)/ 128

In [7]:
#getting the id for images accordingly
img_id = []
for dir in train:
    img_id.append(dir.split("/")[1].split(".")[0])
img_id_df = pd.Series(img_id,name='img_id').astype("int64")


#reshape your image and join
dt1 = d1.reshape([d1.shape[0],32*32*3])
#dt2 = d2.reshape([d2.shape[0],100*100*3])
dt_df = pd.DataFrame(dt1)
#dt_df2 = pd.DataFrame(dt2)

del dt1
#del dt2
del img_id

#joining the dataframe together 
dt = dt_df.join(img_id_df)

del dt_df
#del dt_df2
del img_id_df
gc.collect()

21

In [8]:
#reading the labels datasets
id_df = pd.read_csv("train_labels.csv")
id_df.drop('Unnamed: 0',axis=1,inplace=True)

#merge the two datasets for proper labeling
final_train_df = dt.merge(id_df,on="img_id")

In [9]:
X = final_train_df.drop(['fuel','parking','restaurant','transport','img_id'],axis=1)
y = final_train_df[['fuel','parking','restaurant','transport']]

In [39]:
#train test split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=41)

In [40]:
#converting back to numpy and reshaping
Xtrain = X_train.values.reshape([len(X_train),32,32,3])
Xtest = X_test.values.reshape([len(X_test),32,32,3])
ytrain = y_train.values
ytest = y_test.values

In [76]:
#structure of your convolution layer

model = Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))
model.add(Conv2D(64,(3,3),activation="tanh"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(rate=0.5))
#model.add(Dense(128,activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(4,activation="softmax"))

In [77]:
model.compile("adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [78]:
model.fit(Xtrain,ytrain,batch_size=32,epochs=20)

Epoch 1/20
3015/3015 [==============================] - 11s 4ms/step - loss: 1.3743 - acc: 0.3274
Epoch 2/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.2821 - acc: 0.4017
Epoch 3/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.2355 - acc: 0.4415
Epoch 4/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.1871 - acc: 0.4604
Epoch 5/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.1499 - acc: 0.5041
Epoch 6/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.1172 - acc: 0.5184
Epoch 7/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.0798 - acc: 0.5393A: 4s - loss:  - E
Epoch 8/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.0447 - acc: 0.5648
Epoch 9/20
3015/3015 [==============================] - 8s 3ms/step - loss: 1.0186 - acc: 0.5648
Epoch 10/20
3015/3015 [==============================] - 8s 3ms/step - loss: 0.9942 - acc: 0.5705A: 0s - los

In [60]:
#model.fit(Xtrain,ytrain,batch_size=32,epochs=15)

In [73]:
pred = model.predict(Xtest)

In [74]:
a3 = accuracy_score(ytest.argmax(1),pred.argmax(1))

In [75]:
a3

0.5104477611940299

NameError: name 'Xtest' is not defined

In [8]:
final_train_df.head()

NameError: name 'final_train_df' is not defined